# Notebook para responder a parte 3 do PP1 de RNA

## 3 Tipos de Tarefas (Dayvson)
Recapitulem que dados fornecem experiência sobre um problema. No caso em questão, sugira:
- [x] Uma tarefa de classificação mediante Aprendizado Supervisionado que poderia ser feita com
esta base de dados. Qual seria o atributo-alvo? Quais métricas de desempenho poderiam ser
aplicadas? Que tipo de validação seria apropriado?
- [ ] Uma tarefa de regressão mediante Aprendizado Supervisionado que poderia ser feita com
esta base de dados. Qual seria o atributo-alvo? Quais atributos preditores a equipe considera
relevantes para o cenário?
- [ ] Bônus: Qual tarefa de Aprendizado Não-Supervisionado poderia ser concebida neste contexto?

In [24]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import re


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split

In [30]:
file = '/home/dayvson/disciplinas/rna/Analise-COVID19-PP1-RNA2020.1/dataset/dataset_limpo_covid19_manaus.csv'
df_dataset = pd.read_csv(file)
df_dataset

,idade,sexo,bairro,conclusao,dt_notificacao,tipo_teste
0,25,F,ADRIANOPOLIS,Recuperado,20/03/2020,RT-PCR
1,41,F,BETANIA,Recuperado,27/03/2020,RT-PCR
2,37,M,CIDADE NOVA,Recuperado,27/04/2020,TESTE RÁPIDO - ANTICORPO
3,44,M,SAO JORGE,Recuperado,27/04/2020,TESTE RÁPIDO - ANTICORPO
4,27,F,COLONIA TERRA NOVA,Recuperado,24/04/2020,TESTE RÁPIDO - ANTICORPO
...,...,...,...,...,...,...
6355,14,F,DA PAZ,Recuperado,25/06/2020,TESTE RÁPIDO - ANTÍGENO
6356,5,F,REDENCAO,Recuperado,18/06/2020,TESTE RÁPIDO - ANTÍGENO
6357,70,F,DOM PEDRO I,Recuperado,02/06/2020,TESTE RÁPIDO - ANTÍGENO
6358,68,M,JORGE TEIXEIRA,Recuperado,01/06/2020,RT-PCR


In [31]:
df_dataset.dt_notificacao = pd.to_datetime(df_dataset.dt_notificacao)

## Visualizando alguns valores dos atributos

In [3]:
df_dataset.sexo.value_counts()

F    3605
M    2755
Name: sexo, dtype: int64

In [4]:
df_dataset.conclusao.value_counts()

Recuperado    6347
Óbito           13
Name: conclusao, dtype: int64

In [5]:
df_dataset.tipo_teste.value_counts()

TESTE RÁPIDO - ANTICORPO    3693
RT-PCR                      1521
TESTE RÁPIDO - ANTÍGENO     1139
ECLIA IgG                      4
ELISA IgM                      3
Name: tipo_teste, dtype: int64

> Uma tarefa de classificação mediante Aprendizado Supervisionado que poderia ser feita com esta base de dados. Qual seria o atributo-alvo? Quais métricas de desempenho poderiam ser aplicadas? Que tipo de validação seria apropriado?

## Propondo terefa de classificação para a base de dados

### Tarefa: classificar se o "paciente" se recuperou ou não do covid-19

Com base nos dados solicitados pelo projeto, uma tarefa de classificação que pode ser realizada em cima dos dados é: classificar se um "paciente" se recuperou da covid-19 ou evoluiu para Óbito. Um dos problemas encontrados para realizar esta tarefa é o desbalenceamento de clases, pois enquanto a quantidade de "pacientes" é 6347 o número de pessoas com conclusão igual a Óbito chega a 13.

Uma possível forma de se contornar esse problema  do desbalenceamento das clases seria, selecionar outros atributos, pois observa-se que há bastante pessoas classificadas como Óbito, mas por causa da limpeza e escolha desses atributos do dataset atual essa classe diminuiu considerávelmente.

Outra sugestão para aumentar a quantidade de classes seria considerar mais registros com campos faltantes e depois adicionar a média do atributo para este campo ou outra encontrar outra medida para preencher tais campos.

### Criando um experimento com Árvore de Decisão, Floresta Aletória e Gaussian Naive Bayes para classificar "pacientes"

Foram escolhidos modelos baseados em árvore de decisão e um baseado em probabilidade. A escolha desses algoritmos se deu por primeiro o conjunto de dados não ser tão extenso e por serem eficientes na tarefa de classificar.

In [6]:
X = df_dataset.iloc[:, [0, 1, 2, 4, 5]]
X                       

,idade,sexo,bairro,dt_notificacao,tipo_teste
0,25,F,ADRIANOPOLIS,20/03/2020,RT-PCR
1,41,F,BETANIA,27/03/2020,RT-PCR
2,37,M,CIDADE NOVA,27/04/2020,TESTE RÁPIDO - ANTICORPO
3,44,M,SAO JORGE,27/04/2020,TESTE RÁPIDO - ANTICORPO
4,27,F,COLONIA TERRA NOVA,24/04/2020,TESTE RÁPIDO - ANTICORPO
...,...,...,...,...,...
6355,14,F,DA PAZ,25/06/2020,TESTE RÁPIDO - ANTÍGENO
6356,5,F,REDENCAO,18/06/2020,TESTE RÁPIDO - ANTÍGENO
6357,70,F,DOM PEDRO I,02/06/2020,TESTE RÁPIDO - ANTÍGENO
6358,68,M,JORGE TEIXEIRA,01/06/2020,RT-PCR


In [7]:
y = df_dataset['conclusao']
y

0       Recuperado
1       Recuperado
2       Recuperado
3       Recuperado
4       Recuperado
           ...    
6355    Recuperado
6356    Recuperado
6357    Recuperado
6358    Recuperado
6359    Recuperado
Name: conclusao, Length: 6360, dtype: object

## Teste com holdout

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.shape, y_train.shape

X_test.shape, y_test.shape


clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

ValueError: could not convert string to float: 'F'

## Teste com cross-validation

In [ ]:
Te

In [12]:

scoring = ['precision_macro', 'recall_macro', 'accuracy']
clf = DecisionTreeClassifier()
scores = cross_validate(clf, X, y, scoring=scoring, cv=5, return_estimator=False)
print(sorted(scores.keys()))

['fit_time', 'score_time', 'test_accuracy', 'test_precision_macro', 'test_recall_macro']


array([nan, nan, nan, nan, nan])

- classificar se o paciente é F ou M

- classificar se o paciente foi a Óbito ou se Recuperou